In [1]:
import os
import PIL
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import h5py
import cv2
%matplotlib inline

In [2]:
labels = []

In [3]:
filename = None

for filename in range(1, 3065):
    with h5py.File('./brain_tumor_dataset/imageData/{}.mat'.format(filename), 'r') as f:
        label = f['cjdata']['label'][0][0] - 1
        labels.append(int(label))

In [4]:
for i in range(len(labels)):
    if(labels[i]==0):
        labels[i] = "glioma"
    elif(labels[i]==1):
        labels[i] = "meningioma"
    else:
        labels[i] = "pituitary"

In [5]:
def importing_data(path):
    sample = []
    for filename in os.listdir(path):
        if "jpg" in filename:
            sample.append(filename)
    return sample

In [6]:
path = './brain_tumor_dataset/bt_images/'
train = importing_data(path)

train_data = pd.DataFrame({'image':train, 'label': labels})

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_val = train_test_split(train_data, test_size = 0.1, shuffle = True, random_state = 42)

In [8]:
IMG_SIZE = 64
size = (IMG_SIZE,IMG_SIZE)

In [9]:
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.layers import Dense, Conv2D , MaxPooling2D, Flatten, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Nadam

In [11]:
inputs = Input(shape = (64, 64, 1))
conv1 =  Conv2D(32, (3,3), activation = 'relu', padding = 'same', strides=(1,1), input_shape = [64,64,1])(inputs)
pool1 = MaxPooling2D(2)(conv1)
conv2 = Conv2D(64, (3,3), activation = 'relu', padding = 'same', strides=(1,1))(pool1)
pool2 = MaxPooling2D(2)(conv2)
conv3 = Conv2D(64, (3,3), activation = 'relu', padding = 'same', strides=(1,1))(pool2)
pool3 = MaxPooling2D(2)(conv3)
conv4 = Conv2D(128,(3,3), activation = 'relu', padding = 'same', strides=(1,1))(pool3)
pool4 = MaxPooling2D(2)(conv4)
conv5 = Conv2D(256,(3,3), activation = 'relu', padding = 'same', strides=(1,1))(pool4)
pool5 = MaxPooling2D(2)(conv5)
flatten = Flatten()(pool5)
dense1 = Dense(32, activation = 'relu')(flatten)
dropout1 = Dropout(0.24)(dense1)
outputs = Dense(3, activation = 'softmax')(dropout1)
model = Model(inputs=inputs, outputs=outputs)


model.summary()


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64, 64, 1)]       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 64, 64, 32)        320       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 16, 16, 64)        36928     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 8, 8, 64)          0   

In [12]:
datagen = ImageDataGenerator(rescale = 1./255)

train_set = datagen.flow_from_dataframe(X_train,
                                        directory = './brain_tumor_dataset/bt_images/',
                                        x_col = 'image',
                                        y_col = 'label',
                                        target_size = size,
                                        color_mode = 'grayscale',
                                        class_mode = 'categorical',
                                        batch_size = 1,
                                        shuffle = True,
                                        interpolation = 'bilinear')

val_set = datagen.flow_from_dataframe(X_val,
                                      directory = './brain_tumor_dataset/bt_images/',
                                      x_col = 'image',
                                      y_col = 'label',
                                      target_size = size,
                                      color_mode = 'grayscale',
                                      class_mode = 'categorical',
                                      batch_size = 1,
                                      shuffle = True,
                                      interpolation = 'bilinear')

    

model.compile(optimizer="nadam",loss='categorical_crossentropy',metrics=['accuracy']) 
es = EarlyStopping(monitor='val_loss', mode='min', patience=2, restore_best_weights=True, verbose=1)
checkpoint_cb = ModelCheckpoint("./models/bayesian_relu.h5", save_best_only=True)
history = model.fit(train_set, validation_data = val_set, epochs= 30, callbacks=[checkpoint_cb, es])   

Found 2757 validated image filenames belonging to 3 classes.
Found 307 validated image filenames belonging to 3 classes.
Epoch 1/30
2757/2757 [==============================] - 81s 16ms/step - loss: 0.8140 - accuracy: 0.6449 - val_loss: 0.5576 - val_accuracy: 0.8143
Epoch 2/30
2757/2757 [==============================] - 37s 14ms/step - loss: 0.5262 - accuracy: 0.8049 - val_loss: 0.4874 - val_accuracy: 0.8274.5280 - accuracy - ETA: 1s - l - ETA: 1s - l
Epoch 3/30
2757/2757 [==============================] - 32s 12ms/step - loss: 0.4732 - accuracy: 0.8179 - val_loss: 0.5122 - val_accuracy: 0.8339
Epoch 4/30
2757/2757 [==============================] - 36s 13ms/step - loss: 0.4524 - accuracy: 0.8263 - val_loss: 0.4291 - val_accuracy: 0.8469
Epoch 5/30
2757/2757 [==============================] - 38s 14ms/step - loss: 0.3961 - accuracy: 0.8437 - val_loss: 0.3734 - val_accuracy: 0.8664
Epoch 6/30
2757/2757 [==============================] - 44s 16ms/step - loss: 0.3873 - accuracy: 0.8517 -

In [10]:
def preprocess(img):
    img = img.astype(np.float32)
    img /= 255.0
    img = np.expand_dims(img,0)
    return img

In [11]:
val = X_val["image"].values.tolist()

In [12]:
class_names = {0: "glioma", 1: "meningoma", 2:"pituitary"}
d = 1
iters = 600
popsize = 10
model_path = "./models/bayesian_relu.h5"

In [13]:
import keras 
keras_model = keras.models.load_model(model_path)
keras_model.compile(optimizer = "nadam", loss='categorical_crossentropy', metrics =['accuracy'])

In [14]:
def perturb(x):
    adv_img = orig.copy()

    pixs = np.array(np.split(x, len(x)/3)).astype(int)
    loc = (pixs[:, 0], pixs[:,1])
    val = pixs[:, 2]
    adv_img[loc] = val
    adv_img = np.expand_dims(adv_img,0)
    return adv_img

In [15]:
def optimize(x):
    adv_img = perturb(x)

    prob = keras_model.predict(preprocess(adv_img[0]))
    pred_orig = np.argmax(prob_orig)

    return prob[0][pred_orig]

In [16]:
pred_adv = 0
prob_adv = 0
ct = 0
def callback(x, convergence):
    global pred_adv, prob_adv, ct
    adv_img = perturb(x)


    prob = keras_model.predict(preprocess(adv_img[0]))

    pred_adv = np.argmax(prob)
    prob_adv = prob[0][pred_adv]
    if pred_adv != pred_orig:
        ct = ct+1
        return True

In [20]:
from scipy.optimize import differential_evolution

for filename in val:
    pred_adv = 0
    prob_adv = 0
    orig = cv2.imread(os.path.join("./brain_tumor_dataset/bt_images/",filename))
    orig = cv2.cvtColor(orig, cv2.COLOR_BGR2GRAY)
    orig = cv2.resize(orig, size)
    
    img = orig.copy()
    img = img.reshape(64,64,1)
    img = np.expand_dims(img,0)
    shape = img.shape
    
    
    prob_orig = keras_model.predict(preprocess(img[0]))
    pred_orig = np.argmax(prob_orig)

    bounds = [(0, shape[1]-1), (0, shape[2]-1), (0, 255)] * d
    result = differential_evolution(optimize, bounds, maxiter=iters, popsize=popsize, tol=1e-5, callback=callback)

    adv_img = perturb(result.x)


    prob = keras_model.predict(preprocess(adv_img[0]))
    print('Prob [%s]: %f --> Prob[%s]: %f' %(class_names[pred_orig], prob_orig[0][pred_orig], class_names[pred_adv], prob_adv))

    if(pred_orig != pred_adv):
        cv2.imwrite('./bayesian_outputs_relu/'+filename, adv_img[0])

Prob [pituitary]: 0.998921 --> Prob[pituitary]: 0.613738
Prob [meningoma]: 0.854514 --> Prob[glioma]: 0.515385
Prob [pituitary]: 0.999983 --> Prob[pituitary]: 0.923243
Prob [pituitary]: 0.997020 --> Prob[pituitary]: 0.985843
Prob [pituitary]: 0.998551 --> Prob[pituitary]: 0.917505
Prob [glioma]: 0.999394 --> Prob[meningoma]: 0.459990
Prob [meningoma]: 0.999998 --> Prob[meningoma]: 0.999994
Prob [pituitary]: 0.993747 --> Prob[pituitary]: 0.719615
Prob [glioma]: 0.912210 --> Prob[glioma]: 0.820610
Prob [pituitary]: 0.998415 --> Prob[pituitary]: 0.990984
Prob [pituitary]: 0.632531 --> Prob[meningoma]: 0.844622
Prob [meningoma]: 0.999999 --> Prob[meningoma]: 0.999999
Prob [glioma]: 0.990180 --> Prob[pituitary]: 0.878959
Prob [pituitary]: 0.999523 --> Prob[pituitary]: 0.793559
Prob [meningoma]: 0.999455 --> Prob[meningoma]: 0.993930
Prob [meningoma]: 1.000000 --> Prob[meningoma]: 1.000000
Prob [meningoma]: 0.741368 --> Prob[pituitary]: 0.584294
Prob [meningoma]: 1.000000 --> Prob[meningoma]

Prob [meningoma]: 0.648412 --> Prob[glioma]: 0.547308
Prob [meningoma]: 0.957733 --> Prob[meningoma]: 0.761575
Prob [meningoma]: 0.999954 --> Prob[meningoma]: 0.994823
Prob [pituitary]: 0.604528 --> Prob[meningoma]: 0.567369
Prob [pituitary]: 0.837679 --> Prob[meningoma]: 0.551268
Prob [pituitary]: 0.999588 --> Prob[pituitary]: 0.955177
Prob [pituitary]: 0.999843 --> Prob[pituitary]: 0.940609
Prob [pituitary]: 0.969970 --> Prob[pituitary]: 0.531910
Prob [glioma]: 0.983495 --> Prob[pituitary]: 0.725692
Prob [meningoma]: 0.999423 --> Prob[meningoma]: 0.997256
Prob [glioma]: 0.979361 --> Prob[glioma]: 0.750470
Prob [pituitary]: 0.983218 --> Prob[pituitary]: 0.811263
Prob [meningoma]: 0.951555 --> Prob[pituitary]: 0.639809
Prob [glioma]: 0.999764 --> Prob[glioma]: 0.998798
Prob [pituitary]: 0.996098 --> Prob[pituitary]: 0.761504
Prob [meningoma]: 0.677265 --> Prob[pituitary]: 0.507850
Prob [meningoma]: 0.998687 --> Prob[meningoma]: 0.963060
Prob [meningoma]: 0.960895 --> Prob[meningoma]: 0

Prob [meningoma]: 0.818296 --> Prob[glioma]: 0.500308
Prob [glioma]: 0.383990 --> Prob[meningoma]: 0.539560
Prob [glioma]: 0.987797 --> Prob[glioma]: 0.792511
Prob [pituitary]: 0.987080 --> Prob[pituitary]: 0.636049
Prob [pituitary]: 0.999992 --> Prob[pituitary]: 0.998934
Prob [glioma]: 0.991247 --> Prob[glioma]: 0.911736
Prob [glioma]: 0.992090 --> Prob[glioma]: 0.805726
Prob [pituitary]: 0.999915 --> Prob[pituitary]: 0.976687
Prob [meningoma]: 0.985875 --> Prob[meningoma]: 0.941856
Prob [meningoma]: 0.988320 --> Prob[meningoma]: 0.802342


In [21]:
print(ct)

74


In [17]:
datagen = ImageDataGenerator(rescale = 1./255)
adv_set = datagen.flow_from_dataframe(X_val,
                                      directory = './bayesian_outputs_relu/',
                                      x_col = 'image',
                                      y_col = 'label',
                                      target_size = size,
                                      color_mode = 'grayscale',
                                      class_mode = 'categorical',
                                      batch_size = 1,
                                      shuffle = True,
                                      interpolation = 'bilinear')

D:\Softwares\anaconda\envs\tf-gpu\lib\site-packages\keras_preprocessing\image\dataframe_iterator.py:279: UserWarning: Found 233 invalid image filename(s) in x_col="image". These filename(s) will be ignored.
  warnings.warn(


Found 74 validated image filenames belonging to 3 classes.


In [16]:
predict = keras_model.evaluate(adv_set)

74/74 [==============================] - 62s 62ms/step - loss: 1.1354 - accuracy: 0.2973


In [17]:
path = './bayesian_outputs_relu/'
adv = importing_data(path)
adv_df = pd.DataFrame({'image':adv})
val_df = X_val[~X_val.image.isin(adv_df.image)]

In [18]:
val_set = datagen.flow_from_dataframe(X_val,
                                      directory = './brain_tumor_dataset/bt_images/',
                                      x_col = 'image',
                                      y_col = 'label',
                                      target_size = size,
                                      color_mode = 'grayscale',
                                      class_mode = 'categorical',
                                      batch_size = 1,
                                      shuffle = True,
                                      interpolation = 'bilinear')

Found 307 validated image filenames belonging to 3 classes.


In [19]:
keras_model.evaluate(val_set)

233/233 [==============================] - 6s 26ms/step - loss: 0.1677 - accuracy: 0.9657


[0.16769343614578247, 0.9656652212142944]

In [19]:
es = EarlyStopping(monitor='val_loss', mode='min', patience=2, restore_best_weights=True, verbose=1)

checkpoint_cb = ModelCheckpoint("./models/bayesian_relu_adversarial.h5", save_best_only=True)

history_adv = keras_model.fit(adv_set, validation_data = val_set, epochs= 50, batch_size = 1, callbacks=[es, checkpoint_cb])   

Epoch 1/50
74/74 [==============================] - 73s 276ms/step - loss: 0.9709 - accuracy: 0.6081 - val_loss: 0.1856 - val_accuracy: 0.9479
Epoch 2/50
74/74 [==============================] - 6s 76ms/step - loss: 0.3934 - accuracy: 0.8243 - val_loss: 0.1681 - val_accuracy: 0.9544
Epoch 3/50
74/74 [==============================] - 6s 80ms/step - loss: 0.2529 - accuracy: 0.9054 - val_loss: 0.1567 - val_accuracy: 0.9577
Epoch 4/50
74/74 [==============================] - 5s 74ms/step - loss: 0.1725 - accuracy: 0.9189 - val_loss: 0.1866 - val_accuracy: 0.9414
Epoch 5/50
74/74 [==============================] - 6s 81ms/step - loss: 0.1140 - accuracy: 0.9595 - val_loss: 0.1868 - val_accuracy: 0.9577
Restoring model weights from the end of the best epoch.
Epoch 00005: early stopping


In [20]:
class_names = {0: "glioma", 1: "meningoma", 2:"pituitary"}
d = 1
iters = 600
popsize = 10
model_path = "./models/bayesian_relu_adversarial.h5"
keras_model = keras.models.load_model(model_path)
keras_model.compile(optimizer = "nadam", loss='categorical_crossentropy', metrics =['accuracy'])

In [21]:
from scipy.optimize import differential_evolution

for filename in val:
    pred_adv = 0
    prob_adv = 0
    orig = cv2.imread(os.path.join("./brain_tumor_dataset/bt_images/",filename))
    orig = cv2.cvtColor(orig, cv2.COLOR_BGR2GRAY)
    orig = cv2.resize(orig, size)
    
    img = orig.copy()
    img = img.reshape(64,64,1)
    img = np.expand_dims(img,0)
    shape = img.shape
    
    
    prob_orig = keras_model.predict(preprocess(img[0]))
    pred_orig = np.argmax(prob_orig)

    bounds = [(0, shape[1]-1), (0, shape[2]-1), (0, 255)] * d
    result = differential_evolution(optimize, bounds, maxiter=iters, popsize=popsize, tol=1e-5, callback=callback)

    adv_img = perturb(result.x)

    prob = keras_model.predict(preprocess(adv_img[0]))
    print('Prob [%s]: %f --> Prob[%s]: %f ' %(class_names[pred_orig], prob_orig[0][pred_orig], class_names[pred_adv], prob_adv))

Prob [pituitary]: 0.999423 --> Prob[pituitary]: 0.954439 
Prob [glioma]: 0.987098 --> Prob[meningoma]: 0.890613 
Prob [pituitary]: 0.999859 --> Prob[pituitary]: 0.989252 
Prob [pituitary]: 0.993347 --> Prob[pituitary]: 0.978463 
Prob [pituitary]: 0.997754 --> Prob[pituitary]: 0.960529 
Prob [glioma]: 1.000000 --> Prob[glioma]: 0.999999 
Prob [meningoma]: 0.999999 --> Prob[meningoma]: 0.999998 
Prob [pituitary]: 0.988734 --> Prob[pituitary]: 0.880223 
Prob [glioma]: 0.649121 --> Prob[meningoma]: 0.508525 
Prob [pituitary]: 0.995157 --> Prob[pituitary]: 0.922749 
Prob [meningoma]: 0.527894 --> Prob[pituitary]: 0.518347 
Prob [meningoma]: 0.999999 --> Prob[meningoma]: 0.999994 
Prob [glioma]: 0.999992 --> Prob[glioma]: 0.999964 
Prob [pituitary]: 0.998375 --> Prob[pituitary]: 0.864776 
Prob [meningoma]: 0.998781 --> Prob[meningoma]: 0.990321 
Prob [meningoma]: 1.000000 --> Prob[meningoma]: 1.000000 
Prob [meningoma]: 0.939189 --> Prob[meningoma]: 0.801972 
Prob [meningoma]: 1.000000 --> P

Prob [meningoma]: 1.000000 --> Prob[meningoma]: 1.000000 
Prob [meningoma]: 0.999989 --> Prob[meningoma]: 0.999525 
Prob [pituitary]: 0.999979 --> Prob[pituitary]: 0.996279 
Prob [meningoma]: 0.983074 --> Prob[meningoma]: 0.813169 
Prob [meningoma]: 0.960354 --> Prob[meningoma]: 0.788069 
Prob [meningoma]: 0.999998 --> Prob[meningoma]: 0.999997 
Prob [meningoma]: 0.957479 --> Prob[meningoma]: 0.704874 
Prob [pituitary]: 0.739641 --> Prob[pituitary]: 0.569769 
Prob [pituitary]: 0.996505 --> Prob[pituitary]: 0.947429 
Prob [pituitary]: 0.999493 --> Prob[pituitary]: 0.993720 
Prob [pituitary]: 0.955695 --> Prob[pituitary]: 0.723602 
Prob [glioma]: 0.999976 --> Prob[glioma]: 0.999954 
Prob [meningoma]: 0.998233 --> Prob[meningoma]: 0.945392 
Prob [glioma]: 0.999821 --> Prob[glioma]: 0.991063 
Prob [pituitary]: 0.746365 --> Prob[meningoma]: 0.513337 
Prob [meningoma]: 0.999389 --> Prob[meningoma]: 0.985656 
Prob [glioma]: 0.999399 --> Prob[glioma]: 0.996361 
Prob [pituitary]: 0.980526 --> P

Prob [meningoma]: 0.999999 --> Prob[meningoma]: 0.999991 
Prob [meningoma]: 1.000000 --> Prob[meningoma]: 1.000000 
Prob [meningoma]: 1.000000 --> Prob[meningoma]: 1.000000 
Prob [pituitary]: 0.988004 --> Prob[pituitary]: 0.848269 
Prob [meningoma]: 0.999908 --> Prob[meningoma]: 0.998646 
Prob [meningoma]: 1.000000 --> Prob[meningoma]: 1.000000 
Prob [glioma]: 0.995539 --> Prob[meningoma]: 0.954291 
Prob [meningoma]: 0.906503 --> Prob[meningoma]: 0.566968 
Prob [glioma]: 0.999835 --> Prob[glioma]: 0.996576 
Prob [pituitary]: 0.977374 --> Prob[pituitary]: 0.723654 
Prob [pituitary]: 0.999982 --> Prob[pituitary]: 0.999738 
Prob [glioma]: 0.999756 --> Prob[glioma]: 0.994700 
Prob [glioma]: 0.999753 --> Prob[glioma]: 0.983498 
Prob [pituitary]: 0.999556 --> Prob[pituitary]: 0.980937 
Prob [meningoma]: 0.999238 --> Prob[meningoma]: 0.997099 
Prob [meningoma]: 0.992698 --> Prob[meningoma]: 0.932823 


In [22]:
print(ct)

31


In [27]:
calculate_ssim(adv_img[0], orig)

NameError: name 'calculate_ssim' is not defined